# 03. Pseudocódigos

## 3.3.2 Figura 3.7

Algoritmo de busca pela melhor escolha e função para expandir um nó. As estruturas de dados usadas aqui são descritas na seção 3.3.2 do livro.

```pseudo
function BEST-FIRST-SEARCH(problem, f) returns a solution node or failure
    node ← NODE(STATE = problem.INITIAL)
    frontier ← a priority queue ordered by f, with node as an element
    reached ← a lookup table, with one entry with key problem.INITIAL and value node

    while not IS-EMPTY(frontier) do
        node ← POP(frontier)
        if problem.IS-GOAL(node.STATE) then return node

        for each child in EXPAND(problem, node) do
            s ← child.STATE
            if s is not in reached or child.PATH-COST < reached[s].PATH-COST then
                reached[s] ← child
                add child to frontier

    return failure

function EXPAND(problem, node) yields nodes
    s ← node.STATE
    for each action in problem.ACTIONS(s) do
        s' ← problem.RESULT(s, action)
        cost ← node.PATH-COST + problem.ACTION-COST(s, action, s')
        yield NODE(STATE = s', PARENT = node, ACTION = action, PATH-COST = cost)
```

### Definições

| Símbolo | Descrição |
| :---: | :--- |
| STATE (Estado): | Em qual nó em que o algoritmo de busca está. |
| PARENT (Pai) | Nó pai do nó atual. |
| ACTION (Ação) | Ação que levou ao nó atual. |
| PATH-COST (Custo do caminho) | Custo do caminho do nó inicial até o nó atual. |

Mapa rodoviário simplificado de parte da Romênia 

In [1]:
romania_map = {
    'Arad': {'Zerind': 75, 'Sibiu': 140, 'Timisoara': 118},
    'Zerind': {'Oradea': 71, 'Arad': 75},
    'Oradea': {'Zerind': 71, 'Sibiu': 151},
    'Sibiu': {'Arad': 140, 'Oradea': 151, 'Fagaras': 99, 'Rimnicu Vilcea': 80},
    'Timisoara': {'Arad': 118, 'Lugoj': 111},
    'Lugoj': {'Timisoara': 111, 'Mehadia': 70},
    'Mehadia': {'Lugoj': 70, 'Drobeta': 75},
    'Drobeta': {'Mehadia': 75, 'Craiova': 120},
    'Craiova': {'Drobeta': 120, 'Rimnicu Vilcea': 146, 'Pitesti': 138},
    'Rimnicu Vilcea': {'Sibiu': 80, 'Craiova': 146, 'Pitesti': 97},
    'Fagaras': {'Sibiu': 99, 'Bucharest': 211},
    'Pitesti': {'Craiova': 138, 'Rimnicu Vilcea': 97, 'Bucharest': 101},
    'Bucharest': {'Fagaras': 211, 'Pitesti': 101},
    'Giurgiu': {'Bucharest': 90},
    'Urziceni': {'Bucharest': 85, 'Hirsova': 98, 'Vaslui': 142},
    'Hirsova': {'Urziceni': 98, 'Eforie': 86},
    'Eforie': {'Hirsova': 86},
    'Vaslui': {'Urziceni': 142, 'Iasi': 92},
    'Iasi': {'Vaslui': 92, 'Neamt': 87},
    'Neamt': {'Iasi': 87}
}


`__lt__` e `__gt__` são requisitos da biblioteca heapq.

In [2]:
class Node:
    def __init__(self, state, parent=None, action=None, path_cost=0):
        self.state = state
        self.parent = parent
        self.action = action
        self.path_cost = path_cost

    def __lt__(self, other):
        return self.path_cost < other.path_cost

    def __gt__(self, other):
        return self.path_cost > other.path_cost


Esta classe fez jus ao nome. Me deu muito Problem.

In [3]:
class Problem:
    def __init__(self, initial, goal):
        self.initial = Node(initial)
        self.goal = Node(goal)

    def goal_test(self, state):
        return state == self.goal.state
    
    def actions(self, state):
        return romania_map[state].keys()
    
    def result(self, parent_state, new_state):
        if new_state != parent_state:
            return True
        return False
        
    

Já esta, conseguiu expandir os problemas.

In [4]:
def expand(node, problem):
    current_state = node.state
    children = []
    actions = problem.actions(current_state)

    for action in actions:
        new_state = action
        path_cost = romania_map[current_state][new_state] + node.path_cost
        child = Node(new_state, node, action, path_cost)
        children.append(child)

    return children



Esta função não faz parte do pseudocódigo, adicionei apenas para ter um feedback visual do que está acontecendo. Ela imprime o custo total do caminho e o caminho em si. Neste exercício, estou usando a distância em milhas como custo.

In [5]:
def print_solution(node):
    path = []
    path_cost = 0
    while node.parent:
        path.append(node.state)
        path_cost += node.path_cost
        node = node.parent
    path.append(node.state)
    path.reverse()
    print(f"[{path_cost}] {' -> '.join(path)}")

In [6]:
def heuristic(node, goal):
    current_state = node.state
    goal_state = goal.state
    distances = romania_map

    if current_state in distances.keys():
        if goal_state in distances[current_state].keys():
            distance = distances[current_state][goal_state]
        else:
            distance = float('inf')
    else:
        distance = float('inf')

    return distance


In [7]:
import heapq

def best_first_graph_search(problem, heuristic):
    node = problem.initial
    if problem.goal_test(node.state):
        return node

    frontier = []

    heapq.heappush(frontier, (heuristic(node, problem.goal), node))
    explored = set()

    while frontier:
        node = heapq.heappop(frontier)[1]
        if problem.goal_test(node.state):
            return node

        explored.add(node.state)

        for child in expand(node, problem):
            if child.state not in explored:
                heapq.heappush(frontier, (heuristic(child, problem.goal), child))

    raise ValueError('No solution found')



In [8]:
problem = Problem('Arad', 'Bucharest')
search = best_first_graph_search(problem, heuristic)
print_solution(search)

[1095] Arad -> Sibiu -> Rimnicu Vilcea -> Pitesti -> Bucharest


O algoritmo tem um erro que impede de avançar para o próximo nó, quando chega em `Bucharest`. Não consegui resolver sem gerar um loop infinito. Considerando que o próximo tópico do livro é sobre "Caminhos redundantes", estou considerando que talvez seja um erro proposital. Portanto, ficará assim, até por que já gastei muito tempo neste exercício.